In [1]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
!unzip -q ./XLCoST_data.zip

Downloading...
From (original): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
From (redirected): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6&confirm=t&uuid=5fb7df19-ab8a-44d4-89b9-171483c7afbd
To: /kaggle/working/XLCoST_data.zip
100%|████████████████████████████████████████| 298M/298M [00:03<00:00, 79.4MB/s]


In [2]:
from transformers import RobertaTokenizer, RobertaModel


def get_model():
    model = RobertaModel.from_pretrained('microsoft/codebert-base')
    tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')
    model = model.eval().cuda()
    return model, tokenizer

model, tokenizer = get_model()

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [3]:
import sys
sys.path.append("/kaggle/input/code-search")
from code_search import get_dataset, collate_fn, ContrastiveTrainer

2024-05-30 10:00:08.006280: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 10:00:08.006414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 10:00:08.130662: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
languages = ["C", "PHP", "Java", "C++", "C#", "Javascript", "Python"]
root_path = "/kaggle/working/XLCoST_data"


dataset = get_dataset(root_path=root_path, languages=languages)

dataset

DatasetDict({
    train: Dataset({
        features: ['data_idx', 'query_code', 'relevant_code', 'src_id', 'src_lang', 'trgt_id', 'trgt_lang', 'language'],
        num_rows: 211254
    })
    val: Dataset({
        features: ['data_idx', 'query_code', 'relevant_code', 'src_id', 'src_lang', 'trgt_id', 'trgt_lang', 'language'],
        num_rows: 11784
    })
    test: Dataset({
        features: ['data_idx', 'query_code', 'relevant_code', 'src_id', 'src_lang', 'trgt_id', 'trgt_lang', 'language'],
        num_rows: 21264
    })
})

In [5]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    "contrastive_trainer",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=200,
    num_train_epochs=1,
    evaluation_strategy="no",
    report_to="none",
    remove_unused_columns=False,
    warmup_steps=1000,
    save_strategy="epoch"
)
trainer = ContrastiveTrainer(
    model,
    training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    data_collator=lambda x: collate_fn(x, tokenizer),
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
200,1.786800
400,1.420300
600,1.379900
800,1.352300
1000,1.353800
1200,1.345500
1400,1.345200
1600,1.336900
1800,1.327500
2000,1.325900


TrainOutput(global_step=26407, training_loss=1.3043662527823034, metrics={'train_runtime': 18374.5124, 'train_samples_per_second': 11.497, 'train_steps_per_second': 1.437, 'total_flos': 0.0, 'train_loss': 1.3043662527823034, 'epoch': 1.0})